# Calculating the LSD profiles 

## 0 Packages and data

In [ ]:
cd ..

In [ ]:
!git clone https://github.com/folsomcp/specpolFlow

In [ ]:
!git clone https://github.com/folsomcp/LSDpy

In [ ]:
ls

In [ ]:
cd LSDpy

In [ ]:

import lsdpy


In [ ]:
cd ..

In [ ]:
import specpolFlow as pol
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

import pandas as pd
import itertools

### This gets rid of the warnings in the cells where it is creating more than 20 figures (i.e. 1.2)
#### Not needed for this notebook but could be useful in the future
#import warnings
#warnings.filterwarnings("default", message="More than 20 figures have been opened.")

In [ ]:
import pandas as pd
sheet_id = '1M6y1Wnsrc-w5FjUMfKaSFa_-foIDAaMe8W4lYNWnWyk'
sheet_name = 'Stars'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
StarData=pd.read_csv(url)

sheet_name = 'Observations'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
Observations=pd.read_csv(url)

## 0.1 Demo with a single observation


In [ ]:
#star = 'hd6226'
#obs = '1'
#modelcode = 'T15000G40'
## careful, check whether it is g or G in the spreadsheet....
#vsini = 120.0
#vradCorrected = -70.0 *u.km/u.s

#folderMask='depth0.1_tellClean'

#obsFolder='Synth-hybrid-spectra'
#obsnamecode = '_noise'

#lsdfolder = 'hybrid__maskdepth0.1_tellClean'

#mask = 'Masks/{}/{}_depth0.1.mask'.format(folderMask,modelcode)

#file = '{}/{}_{}{}.s'.format(obsFolder, star, obs, obsnamecode)
#outfile = 'LSD/{}/{}_{}.lsd'.format(lsdfolder, star,obs)


#toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
#           velStart=-400.0, velEnd=400.0, velPixel=2.6, 
#           normDepth=0.2, normLande=1.2, normWave=500.0,
#           removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
#           interpMode=1, fSaveModelS=0, outModelName='',
#           fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')


In [ ]:
#lsd = pol.iolsd.read_lsd(outfile)
#lsd.plot()

### 2.1 **Spectra**: Synth-hybrid-spectra
reran - Patrick


In [ ]:
df = pd.read_csv('03-LSD-calculations/LSD/Params-LSDProfiles-CalculatedByPython.csv')
blend=np.array(df['Name'][df['VelEnd']>=900])
blend

In [ ]:
Observations

#### **Mask**: tellClean Mask depth 0.02:

In [ ]:
%%capture
with PdfPages('03-LSD-calculations/hybrid__maskdepth0.02_tellClean.pdf') as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    for j in range(0,len(blend)):
      if blend[j]==star:
        mask = '02-Mask-calculations/MaskCleaning/{}_depth0.02.mask'.format(star)
        break
      if blend[j]!=star:
        mask = '02-Mask-calculations/MaskCleaning/depth0.02_tellClean/{}_depth0.02.mask'.format(modelcode)

    file = '01-Synth-calculations/Synth-hybrid-spectra/{}_{}_noise.s'.format(star, obs)
    outfile = '03-LSD-calculations/LSD/hybrid_maskdepth0.02_tellClean/{}_{}.lsd'.format(star,obs)

    toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
            velStart=velS, velEnd=velE, velPixel=velP, 
            normDepth=0.2, normLande=1.2, normWave=500.0,
            removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
            interpMode=1, fSaveModelS=0, outModelName='',
            fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')
    lsd = pol.iolsd.read_lsd(outfile)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean Mask depth 0.1:

In [ ]:
%%capture
with PdfPages('03-LSD-calculations/hybrid__maskdepth0.1_tellClean.pdf') as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    for j in range(0,len(blend)):
      if blend[j]==star:
        mask = '02-Mask-calculations/MaskCleaning/{}_depth0.1.mask'.format(star)
        break
      if blend[j]!=star:
        mask = '02-Mask-calculations/MaskCleaning/depth0.1_tellClean/{}_depth0.1.mask'.format(modelcode)

    file = '01-Synth-calculations/Synth-hybrid-spectra/{}_{}_noise.s'.format(star, obs)
    outfile = '03-LSD-calculations/LSD/hybrid_maskdepth0.1_tellClean/{}_{}.lsd'.format(star,obs)

    toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
            velStart=velS, velEnd=velE, velPixel=velP, 
            normDepth=0.2, normLande=1.2, normWave=500.0,
            removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
            interpMode=1, fSaveModelS=0, outModelName='',
            fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')
    lsd = pol.iolsd.read_lsd(outfile)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean CleanAsifTweak

In [ ]:
%%capture
with PdfPages('03-LSD-calculations/hybrid_CleanAsifTweak_tellClean.pdf') as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        vsini = StarData["Adopted-vsini"][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    mask = '02-Mask-calculations/MaskCleaning/AsifMaskCleanTweak_tellClean/{}_clean_{}.mask_auto_twk'.format(star,obs)

    file = '01-Synth-calculations/Synth-hybrid-spectra/{}_{}_noise.s'.format(star, obs)
    outfile = '03-LSD-calculations/LSD/hybrid_CleanAsifTweak+tellClean/{}_{}.lsd'.format(star,obs)

    toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
            velStart=velS, velEnd=velE, velPixel=velP, 
            normDepth=0.2, normLande=1.2, normWave=500.0,
            removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
            interpMode=1, fSaveModelS=0, outModelName='',
            fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')
    lsd = pol.iolsd.read_lsd(outfile)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean AsifClean

In [ ]:
%%capture
with PdfPages('03-LSD-calculations/hybrid_CleanAsif_tellClean.pdf') as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        vsini = StarData["Adopted-vsini"][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    mask = '02-Mask-calculations/MaskCleaning/AsifMaskClean_tellClean/{}_clean_{}.mask'.format(star,obs)

    file = '01-Synth-calculations/Synth-hybrid-spectra/{}_{}_noise.s'.format(star, obs)
    outfile = '03-LSD-calculations/LSD/hybrid_CleanAsif+tellClean/{}_{}.lsd'.format(star,obs)

    toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
            velStart=velS, velEnd=velE, velPixel=velP, 
            normDepth=0.2, normLande=1.2, normWave=500.0,
            removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
            interpMode=1, fSaveModelS=0, outModelName='',
            fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')
    lsd = pol.iolsd.read_lsd(outfile)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

### 2.2 **Spectra**: Normalized-Spectra
reran - Patrick


In [ ]:
df = pd.read_csv('03-LSD-calculations/LSD/Params-LSDProfiles-CalculatedByPython.csv')
blend=np.array(df['Name'][df['VelEnd']>=900])
blend

#### **Mask**: tellClean Mask depth 0.02:

In [ ]:
%%capture

with PdfPages('03-LSD-calculations/normalized_maskdepth0.02_tellClean.pdf') as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    for j in range(0,len(blend)):
      if blend[j]==star:
        mask = '02-Mask-calculations/MaskCleaning/{}_depth0.02.mask'.format(star)
        break
      if blend[j]!=star:
        mask = '02-Mask-calculations/MaskCleaning/depth0.02_tellClean/{}_depth0.02.mask'.format(modelcode)

    file = '00-InputMaterial/NormalizedSpectra/{}_{}.s'.format(star, obs)
    outfile = '03-LSD-calculations/LSD/normalized_maskdepth0.02_tellClean/{}_{}.lsd'.format(star,obs)

    toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
            velStart=velS, velEnd=velE, velPixel=velP, 
            normDepth=0.2, normLande=1.2, normWave=500.0,
            removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
            interpMode=1, fSaveModelS=0, outModelName='',
            fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')
    lsd = pol.iolsd.read_lsd(outfile)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean Mask depth 0.1:

In [ ]:
%%capture

with PdfPages('03-LSD-calculations/normalized_maskdepth0.1_tellClean.pdf') as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        if StarData['vsini-estimate'][j]=='very bad':
          vsini = StarData["Adopted-vsini"][j]
        else:
          vsini = StarData['vsini-estimate'][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    for j in range(0,len(blend)):
      if blend[j]==star:
        mask = '02-Mask-calculations/MaskCleaning/{}_depth0.1.mask'.format(star)
        break
      if blend[j]!=star:
        mask = '02-Mask-calculations/MaskCleaning/depth0.1_tellClean/{}_depth0.1.mask'.format(modelcode)

    file = '00-InputMaterial/NormalizedSpectra/{}_{}.s'.format(star, obs)
    outfile = '03-LSD-calculations/LSD/normalized_maskdepth0.1_tellClean/{}_{}.lsd'.format(star,obs)

    toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
            velStart=velS, velEnd=velE, velPixel=velP, 
            normDepth=0.2, normLande=1.2, normWave=500.0,
            removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
            interpMode=1, fSaveModelS=0, outModelName='',
            fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')
    lsd = pol.iolsd.read_lsd(outfile)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean CleanAsifTweak

In [ ]:
%%capture
with PdfPages('03-LSD-calculations/normalized_CleanAsifTweak_tellClean.pdf') as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        vsini = StarData["Adopted-vsini"][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    mask = '02-Mask-calculations/MaskCleaning/AsifMaskCleanTweak_tellClean/{}_clean_{}.mask_auto_twk'.format(star,obs)

    file = '00-InputMaterial/NormalizedSpectra/{}_{}.s'.format(star, obs)
    outfile = '03-LSD-calculations/LSD/normalized_CleanAsifTweak+tellClean/{}_{}.lsd'.format(star,obs)

    toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
            velStart=velS, velEnd=velE, velPixel=velP, 
            normDepth=0.2, normLande=1.2, normWave=500.0,
            removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
            interpMode=1, fSaveModelS=0, outModelName='',
            fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')
    lsd = pol.iolsd.read_lsd(outfile)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min

#### **Mask**: tellClean AsifClean

In [ ]:
%%capture
with PdfPages('03-LSD-calculations/normalized_CleanAsif_tellClean.pdf') as pdf:

  for i, ObsInfo in Observations.iterrows():
    star = ObsInfo["Name"]
    obs = ObsInfo["Unnamed: 2"]
    for j in range(0,len(StarData["Name"])):
      if StarData["Name"][j] == star:
        modelcode = StarData["ModelCode"][j].upper()
        vsini = StarData["Adopted-vsini"][j]
        velS = df["VelStart"][j]
        velE = df["VelEnd"][j]
        velP = df["VelPixel"][j]

    mask = '02-Mask-calculations/MaskCleaning/AsifMaskClean_tellClean/{}_clean_{}.mask'.format(star,obs)

    file = '00-InputMaterial/NormalizedSpectra/{}_{}.s'.format(star, obs)
    outfile = '03-LSD-calculations/LSD/normalized_CleanAsif+tellClean/{}_{}.lsd'.format(star,obs)

    toto = lsdpy.main(observation=file, mask=mask, outName=outfile, 
            velStart=velS, velEnd=velE, velPixel=velP, 
            normDepth=0.2, normLande=1.2, normWave=500.0,
            removeContPol=1, trimMask=0, sigmaClipIter=0, sigmaClip=500, 
            interpMode=1, fSaveModelS=0, outModelName='',
            fLSDPlotImg=0, fSavePlotImg=0, outPlotImgName='')
    lsd = pol.iolsd.read_lsd(outfile)
    
    fig, ax = plt.subplots(4,1, figsize=(10,10))

    ax[3].errorbar(lsd.vel, lsd.specI, yerr=lsd.specSigI, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[2].errorbar(lsd.vel, lsd.specN1, yerr=lsd.specSigN1, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[1].errorbar(lsd.vel, lsd.specN2, yerr=lsd.specSigN2, xerr=None, fmt='o', ms=3,ecolor='0.5',c='k')
    ax[0].errorbar(lsd.vel, lsd.specV, yerr=lsd.specSigV, xerr=None, fmt='o', ms=3, ecolor='0.5',c='k')
    plt.subplots_adjust(hspace=.0)
    ax[3].set_xlabel('Velocity (km/s)')
    ax[3].set_ylabel('I')
    ax[2].set_ylabel('N1')
    ax[1].set_ylabel('N2')
    ax[0].set_ylabel('V')
    fig.suptitle(star + '-'+ str(obs),y=0.92)

    pdf.savefig(fig)

##~30min